In [1]:
from models import*

Using TensorFlow backend.


In [2]:
import tensorflow as tf
import keras.backend as K
import os
import numpy as np
import genomelake
from genomelake.extractors import ArrayExtractor
from pybedtools import BedTool
from io_utils import batch_iterator
from sklearn import metrics
from keras.layers import Input

In [3]:
DEFER_DELETE_SIZE=int(250 * 1e6) 
def create_tensorflow_session(visiblegpus):
    os.environ['CUDA_VISIBLE_DEVICES'] = str(visiblegpus)
    session_config = tf.ConfigProto()
    session_config.gpu_options.deferred_deletion_bytes = DEFER_DELETE_SIZE
    session_config.gpu_options.per_process_gpu_memory_fraction = 0.2
    session = tf.Session(config=session_config)
    K.set_session(session)
    return session


In [4]:
sess=create_tensorflow_session(1)


In [5]:
SeqClassifier=SequenceClassifier()
SeqMethClassifier=SequenceMethylationClassifier()
SeqDnaseClassifier=SequenceDNAseClassifier()

models.py:111: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=1)`
  seq_preds = Dense(output_dim=num_tasks)(seq_preds)
models.py:139: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  logits = Merge(mode='concat', concat_axis=2)([seq_preds, methylation_preds])
models.py:225: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  logits = Merge(mode='concat', concat_axis=-1)([seq_preds, dnase_preds])


In [6]:
SeqDnaseClassifier.inputs['data/dnase_data_dir']==dnase_placeholder

True

In [7]:
SeqDnaseClassifier.inputs

OrderedDict([('data/genome_data_dir',
              <tf.Tensor 'input_1:0' shape=(?, 1000, 4) dtype=float32>),
             ('data/dnase_data_dir',
              <tf.Tensor 'input_3:0' shape=(?, 1000, 1) dtype=float32>)])

In [8]:
batch_size=128
num_epochs=10

In [9]:
logits=SeqDnaseClassifier.get_logits()
labels=tf.placeholder(tf.float32,shape=(None,1))
loss=tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=labels,logits=logits))
optimizer=tf.train.AdamOptimizer(learning_rate=.0003)
train_op=optimizer.minimize(loss)





In [10]:
train_data='data/train_intervals.bed'
train_labels='data/train_labels.npy'
validation_data='data/validation_intervals.bed'
validation_labels='data/validation_labels.npy'


In [11]:
genome_file_path='/srv/scratch/manyu/Baseline_TF_binding_models/extracted_data/GRCh38.p3.genome.fa/'

In [12]:
extracted_genome=ArrayExtractor(genome_file_path)

In [13]:
train_data_bedtool=BedTool(train_data)
validation_data_bedtool=BedTool(validation_data)

In [14]:
train_label_vals=np.load(train_labels)
train_label_vals=np.reshape(train_label_vals,(len(train_label_vals,)))
validation_label_vals=np.load(validation_labels)

In [15]:
interval_batch=[train_data_bedtool[_] for _ in range(0,128)]
label_batch=train_label_vals[:128]
extracted_batch=extracted_genome(interval_batch)
dnase_batch=np.expand_dims(extracted_batch[:,:,0],axis=-1)

In [16]:
feed_dict={dnase_placeholder:dnase_batch,seq_placeholder:extracted_batch,labels:label_batch.reshape((len(label_batch),1))}
init_op = tf.global_variables_initializer()
sess.run(init_op)

In [17]:
for i in range(100):
    print(sess.run([loss,train_op],feed_dict=feed_dict))

[0.7788515, None]
[0.6687298, None]
[0.6852591, None]
[0.6742066, None]
[0.66372895, None]
[0.661337, None]
[0.6630584, None]
[0.6632564, None]
[0.66050184, None]
[0.6562425, None]
[0.6523015, None]
[0.65013075, None]
[0.6492052, None]
[0.64615536, None]
[0.6404654, None]
[0.6345513, None]
[0.6298615, None]
[0.6243867, None]
[0.6155157, None]
[0.6058792, None]
[0.59804523, None]
[0.58698124, None]
[0.5742967, None]
[0.56246495, None]
[0.5458544, None]
[0.5304446, None]
[0.51076496, None]
[0.4939952, None]
[0.47551024, None]
[0.45573437, None]
[0.4369752, None]
[0.42151907, None]
[0.42545533, None]
[0.41805, None]
[0.3714412, None]
[0.38996652, None]
[0.35273665, None]
[0.3448971, None]
[0.32916915, None]
[0.31327614, None]
[0.30883956, None]
[0.28546965, None]
[0.28238416, None]
[0.27084774, None]
[0.24656029, None]
[0.24459088, None]
[0.22871093, None]
[0.20924348, None]
[0.20643999, None]
[0.19588771, None]
[0.17969464, None]
[0.16845442, None]
[0.1508508, None]
[0.14399186, None]
[0

In [47]:
weights=SeqDnaseClassifier.model.get_weights()

In [57]:
sess.run(tf.global_variables_initializer())

In [58]:
1

1

In [61]:
A = np.arange(8).reshape((2,2,2))
A

array([[[0, 1],
        [2, 3]],

       [[4, 5],
        [6, 7]]])

In [63]:
A=np.random.random((4,1000))

In [65]:
A_=np.flip(np.flip(A,axis=0),axis=1)
B_=np.flip(np.flip(A,axis=1),axis=0)

In [70]:
C=np.random.random((10,1000,4))

In [71]:
rev_comp_c=C[:,::-1,::-1]

In [74]:
A=np.arange(100).reshape((10,10))

In [25]:
C_mask=extracted_batch[:,:,1].reshape((128,1000,1))

In [26]:
C_mask.shape

(128, 1000, 1)

In [23]:
methylation=np.random.random((128,1000,1))

In [30]:
masked_methylation=methylation*C_mask

In [33]:
from keras import backend as K

In [34]:
K.reverse

<function keras.backend.tensorflow_backend.reverse>

In [55]:
seq_preds=Input(shape=(10,4))
methylation_preds=Input(shape=(10,1))
reverse_comp_seq=K.reverse(K.reverse(seq_preds,axes=-1),axes=-2)
reverse_methylation=K.reverse(methylation_preds,axes=-2)
C_locations_forward_strand=K.expand_dims(seq_preds[:,:,1],axis=-1)
C_locations_reverse_strand=K.expand_dims(reverse_comp_seq[:,:,1],axis=-1)
forward_methylation_masked=methylation_preds*C_locations_forward_strand   #Using the mask that is C
reverse_methylation_masked=reverse_methylation*C_locations_reverse_strand  #Using the reverse comp mask on C

In [44]:
C_locations_reverse_strand


<tf.Tensor 'ExpandDims_5:0' shape=(?, 10, 1) dtype=float32>

In [59]:
def convert_string_to_array(string):
    map_={'a':0,'c':1,'g':2,'t':3}
    to_return=np.zeros((len(string),4))
    for _ in range(len(string)):
        to_return[_,map_[string[_]]]=1
    return np.expand_dims(to_return,axis=0)    
       

In [60]:
sequence=convert_string_to_array('acgcgccgtt')
methylation=np.reshape(np.array([0,0.5,0.5,0.6,0.7,0,0.6,0.7,0,0]),(1,10,1))

In [61]:
feed_dict={seq_preds:sequence,methylation_preds:methylation}

In [63]:
rev_comp_seq,rev_meth,C_forw,C_rev,fw_mask_meth,rev_mask_meth=sess.run([reverse_comp_seq,reverse_methylation,C_locations_forward_strand,C_locations_reverse_strand,
          forward_methylation_masked,reverse_methylation_masked],feed_dict=feed_dict)

In [85]:
def convert_array_to_string(array):
    array=np.squeeze(array,axis=0)
    map_={0:'a',1:'c',2:'g',3:'t'}
    string=[]
    for i in range(len(array)):
        string.append(map_[np.argmax(array[i])])
    return ''.join(string)   
        
    

In [88]:
convert_array_to_string(rev_comp_seq)

'aacggcgcgt'

In [6]:
from keras import Input

In [26]:
a=Input((37,50))

In [18]:
layer_shape=K.get_variable_shape(a)
flattened_shape=layer_shape[1]*layer_shape[2]
Reshape((flattened_shape,))(a)

In [21]:
from keras.layers import Reshape


In [27]:
Reshape((flattened_shape,))(a)

<tf.Tensor 'reshape_4/Reshape:0' shape=(?, 1850) dtype=float32>